Word Mover Distance
![icon]('/data/users/jiancheng.zhai/project/wmd/doc/wmd-obama.png')

In [77]:
import os
os.chdir('/data/users/jiancheng.zhai/project/wmd/doc')
import sys
sys.path.append("../utils")
sys.path.append("../word_movers_distance")
import w2v_embeddings
import WMD_pyemd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
import read_conf
import pandas as pd

In [68]:
word2vec_train_cmd = """python ../bin/word2vec.py ../data/raw_data ../data/raw.model ../data/raw.vector"""
read_conf.exec_cmd(word2vec_train_cmd)

In [70]:
if not os.path.exists("../data/embed.dat"):
    print("Caching word embeddings in memmapped format...")
    from gensim.models.word2vec import Word2Vec
    wv = Word2Vec.load_word2vec_format("../data/raw.vector",  binary=True)
    print "wv syn0norm shape : " + str(wv.syn0norm.shape)
    fp = np.memmap("../data/embed.dat", dtype=np.double, mode='w+', shape=wv.syn0norm.shape)
    fp[:] = wv.syn0norm[:]
    with open("../data/embed.vocab", "w") as f:
        for _, w in sorted((voc.index, word) for word, voc in wv.vocab.items()):
            f.write(w + "\n")
    del fp, wv



Caching word embeddings in memmapped format...
wv syn0norm shape : (1774, 400)


In [71]:
W = np.memmap("../data/embed.dat", dtype=np.double, mode="r", shape=(1774, 400))
#W = np.memmap("../data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("../data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())
#type(wv)
#wv.similarity
#fp = np.memmap("../data/embed.dat", dtype=np.double, mode='w+', shape=wv.syn0norm.shape)

In [64]:
vocab_list

['juxiangyou.com',
 'game.juxiangyou.com',
 'eloancn.com',
 'pangpangpig.com',
 'dhqjr.com',
 'hx9999.com',
 'cgtz.com',
 'yrw.com',
 'app.etongdai.com',
 'touna.cn',
 '91jin.com',
 '99fan.com',
 'wsloan.com',
 'haochedai.com',
 'user.lu.com',
 'gaosouyi.com',
 'trjcn.com',
 'frbao.com',
 'jfz.com',
 'itoumi.com',
 'zhitouwang.cn',
 'yuntongxun.com',
 'huiyingdai.com',
 'cnaidai.com',
 'chenghuitong.net',
 'yinker.com',
 'yinhu.com',
 'old.jinshangdai.com',
 'zhaoshangdai.com',
 'xinhehui.com',
 'hrcf168.com',
 'etongdai.com',
 'edai.com',
 'haoinvest.com',
 'baijindai.com',
 'd.com.cn',
 '9rjr.com',
 'xrkdai.com',
 'ma.juxiangyou.com',
 'lejinsuo.com',
 '360taihe.com',
 '1001.cn',
 'sslj8.cn',
 'qtyd.com',
 'jimengcaifu.com',
 'hitouba.com',
 'gkkxd.com',
 'tuandai.com',
 'fanli.juxiangyou.com',
 'fa.163.com',
 'v.51juejinjie.com',
 'ftoul.com',
 'bbs.wsloan.com',
 'fuyph.com',
 'yibaijin.com',
 'xiangmu.trjcn.com',
 'jia16.com',
 'xiaozhu168.com',
 'bjdp2p.com',
 'fangyuano.com',
 'q

In [2]:
import word_movers_knn

In [72]:
#d1 = "Obama speaks to the media in Illinois"
#d2 = "The President addresses the press in Chicago"
d1 = "app_etongdai_com touna_cn 99fan_com haochedai_com"
d2 = "yuntongxun_com huiyingdai_com yinhu_com zhaoshangdai_com"

vect = CountVectorizer(stop_words='english').fit([d1, d2])
print("Features:",  ", ".join(vect.get_feature_names()))

('Features:', u'99fan_com, app_etongdai_com, haochedai_com, huiyingdai_com, touna_cn, yinhu_com, yuntongxun_com, zhaoshangdai_com')


In [73]:
from scipy.spatial.distance import cosine
v_1, v_2 = vect.transform([d1, d2])
v_1 = v_1.toarray().ravel()
v_2 = v_2.toarray().ravel()
print(v_1, v_2)
print("cosine(doc_1, doc_2) = {:.2f}".format(cosine(v_1, v_2)))

(array([1, 1, 1, 0, 1, 0, 0, 0]), array([0, 0, 0, 1, 0, 1, 1, 1]))
cosine(doc_1, doc_2) = 1.00


In [ ]:
vocab_dict = {w: k for k, w in enumerate(vocab_list)}

from sklearn.metrics import euclidean_distances
W_ = W[[vocab_dict[w] for w in vect.get_feature_names()]]
D_ = euclidean_distances(W_)
print("d(99fan_com, zhaoshangdai_com) = {:.2f}".format(D_[0, 7]))
print("d(99fan_com, app_etongdai_com) = {:.2f}".format(D_[0, 1]))

In [75]:
from pyemd import emd

# pyemd needs double precision input
v_1 = v_1.astype(np.double)
v_2 = v_2.astype(np.double)
v_1 /= v_1.sum()
v_2 /= v_2.sum()
D_ = D_.astype(np.double)
D_ /= D_.max()  # just for comparison purposes
print("d(doc_1, doc_2) = {:.2f}".format(emd(v_1, v_2, D_)))

d(doc_1, doc_2) = 0.84


In [100]:
model = Word2Vec.load("../data/raw.model")
model
s1 = "app_etongdai_com app_etongdai_com app_etongdai_com app_etongdai_com app_etongdai_com 99fan_com haochedai_com".split()   
s2 = "yuntongxun_com huiyingdai_com yinhu_com zhaoshangdai_com".split()
D = WMD_pyemd.WMD(s1, s2, model)
#print D
s1

['app_etongdai_com',
 'app_etongdai_com',
 'app_etongdai_com',
 'app_etongdai_com',
 'app_etongdai_com',
 '99fan_com',
 'haochedai_com']

In [113]:

comp_domain_file = "../data/comp_url_data"
comp_dict = {}

fd = open(comp_domain_file)
for line in fd:
    line.strip()
    arr = line.split('\t')
    comp_id = arr.pop(0)
    comp_dict[comp_id] = arr
fd.close()




In [140]:
s1 = comp_dict['3926']
s2 = comp_dict['4210']
#s1 = [x for x in s1 if x in vocab_list]
#s2 = [x for x in s2 if x in vocab_list]


#D = WMD_pyemd.WMD(s1, s2, model)


In [152]:

#len(s1)
#len(s2)
#print D
#len(s1)
#vocab_list
#aaa = [x for x in s1 if x in vocab_list]
#len(aaa)

comp_list = comp_dict.keys()

for comp_id in comp_list:
    s = comp_dict[comp_id]
    s = [x for x in s if x in vocab_list]
    comp_dict[comp_id] = s
    if(len(s) < 100000):
        del comp_dict[comp_id]

print len(comp_dict.keys())

51


In [ ]:
comp_sim_file = '../data/comp_sim'
wfd = open(comp_sim_file, 'w+')

    
count = 0
while(len(comp_list)>0):
    comp1 = comp_list.pop(0)
    for comp2 in comp_list:
        count += 1
        s1 = comp_dict[comp1]
        s2 = comp_dict[comp2]
#        s1 = [x for x in s1 if x in vocab_list]
#        s2 = [x for x in s2 if x in vocab_list]
#        if(len(s1) < 10000 or len(s2) < 10000):
#            continue
#        print "len pairs :" + str(len(s1)) + "\t" + str(len(s2))

        D = WMD_pyemd.WMD(s1, s2, model)
        
#        print "D :"  + str(D)
        
        #beta = 1.0
        #similarity = np.exp(-beta * D / np.std(D))
        #print str(count) + " => sim : " + str(similarity)
        
        res = comp1 + "\t" + comp2 + "\t" + str(D)
        print res
        wfd.write(res + "\n")
        
wfd.close()

1705	1127	5.59109191961
1705	2543	5.90731191086
1705	1232	5.51730244595
1705	1845	5.54868025808
1705	4101	5.59041360568
1705	4105	6.8166634858
1705	1178	5.8720250124
1705	1572	4.87119705575
1705	1570	4.63662841698
1705	4986	5.85258649795
1705	4038	5.73179614191
1705	3843	5.76013451979
1705	5495	6.05508912309
1705	2272	6.09200731493
1705	2936	5.35139253884
1705	1209	5.59742247902
1705	1514	5.66814940659
1705	2740	6.59699547163
1705	3222	6.32180997003
1705	2320	6.73687567027
1705	4561	9.65311312838
1705	2392	6.54966660834
1705	2257	6.01710740676
1705	2772	6.02924996397
1705	3951	6.43756621467
1845	2527	4.79259810733
1845	3122	5.01205728941
1845	3405	6.03117189036
1845	3448	4.16798207348
1845	4120	4.66772321977
1845	2936	5.24164956892
1845	1209	4.66096682351
1845	1514	5.55474492633
1845	2740	6.68040484909
1845	3222	5.67788946465
1845	2320	5.27809466269
1845	4561	8.2551991304
1845	2392	5.328509629
1845	2257	5.38084780368
1845	2772	4.60619750399
1845	5155	4.74177850914
1845	3811	5.551553706

In [ ]:
dist_matrix_file = "../data/dist_matrix_data"
dist_matrix = pd.read_csv(dist_matrix_file, header=False, sep="\t")

